In [1]:
import os, os.path
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from PIL import Image, ImageDraw, ImageOps

In [2]:
data_folder = "a836e596-d0dd-4eb3-92c6-fddb6c338fb4.nosync/"
RGB_image_path = data_folder + "RGBcf17cfc3-1002-4f38-9824-d3615a73a1cc/"
instance_segmentation_path = data_folder + "InstanceSegmentationc6cef299-4a0f-4d7b-b19c-202699af2eb8/"
backgrounds_videos = "background_videos/"

blur = 3
frame_number = 19
size = 288, 384
fps = 30

In [ ]:
if not os.path.exists('dataset/Videos'):
    os.makedirs('dataset/Videos')
    
if not os.path.exists('dataset/InstanceSegmentation'):
    os.makedirs('dataset/InstanceSegmentation')

path, dirs, files = next(os.walk(RGB_image_path))
file_count = len(files)

path, dirs, files = next(os.walk(backgrounds_videos))

final_images = []
final_segmentations = []

video_count = 0
i = 1
background_video = cv2.VideoCapture(backgrounds_videos + random.choice(files))
for count in range(2, file_count+2):
    if count%100 == 0:
        print(count/frame_number)
    try:
        rgb_image = cv2.imread(RGB_image_path + "rgb_{}.png".format(count))
        segmentation_image = cv2.imread(instance_segmentation_path + "Instance_{}.png".format(count))

        ret, background_image = background_video.read()
        background_image = cv2.cvtColor(background_image, cv2.COLOR_BGR2GRAY)

        subtract = int((rgb_image.shape[1] - ((background_image.shape[1]/background_image.shape[0])*rgb_image.shape[0]))/2)

        u = np.array([0, 0, 0])
        l = np.array([0, 0, 0])

        mask = cv2.inRange(segmentation_image, l, u)
        mask = cv2.blur(mask, (blur, blur)) # Maybe change to use guarsian filter
        if subtract != 0:
            mask = mask[:,subtract:-subtract]
        mask = cv2.resize(mask, (384, 288))

        rgb_image = cv2.blur(rgb_image, (blur, blur)) # Maybe change to use guarsian filter
        if subtract != 0:
            rgb_image = rgb_image[:,subtract:-subtract]
        rgb_image = cv2.resize(rgb_image, (384, 288))

        final = np.array(background_image * (mask/255) + cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY) * (np.ones((288, 384)) - (mask/255)), dtype='uint8')

        if subtract != 0:
            final_segmentation_image = segmentation_image[:,subtract:-subtract]
        else:
            final_segmentation_image = segmentation_image
        final_segmentation_image = cv2.resize(final_segmentation_image, (384, 288))

        final_images.append(final)
        final_segmentations.append(final_segmentation_image)

        i += 1

        if (count-1) % frame_number == 0:
            background_video = cv2.VideoCapture(backgrounds_videos + random.choice(files))

            out = cv2.VideoWriter('dataset/Videos/video_{}.mp4'.format(video_count), cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
            for frame in final_images:
                out.write(frame)
            out.release()

            out = cv2.VideoWriter('dataset/InstanceSegmentation/segmentation_{}.mp4'.format(video_count), cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), True)
            for frame in final_segmentations:
                out.write(frame)
            out.release()

            final_images = []
            final_segmentations = []

            video_count += 1
    except:
        background_video = cv2.VideoCapture(backgrounds_videos + random.choice(files))
        print("fucking shit fuck error: {} {}".format(count))

5.2631578947368425
10.526315789473685
15.789473684210526
21.05263157894737
26.31578947368421
31.57894736842105
36.8421052631579
42.10526315789474
